In [1]:
import numpy as np
import pandas as pd
import goodreads_api_client as gr
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import requests as req
import requests

from urllib.request import urlopen
import re

import time
import sys

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### Getting books and reviews directly from goodreads.

In [3]:
from importlib import reload  
import src.helper_functions as hf 
hf = reload(hf)

In [4]:
# b_links = []

# page = pages[25]
# r = requests.get(page)
# soup = bs(r.content, 'html')
# divs = soup.find_all('div', {"class": "elementList"})[:50]
# links_for_books = ['http://goodreads.com/' + divs[i].find_all("a", {'class':"bookTitle"})[0].get('href') for i in range(50)]
# b_links.extend(links_for_books)


In [78]:
popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science?page=1'
pages = [f'https://www.goodreads.com/shelf/show/popular-science?page={x}' for x in range(1, 26)]

In [4]:
def gr_sign_in(driver):
    login_page = 'https://www.goodreads.com'
    driver.get(login_page)

    username = driver.find_element_by_id("userSignInFormEmail")
    username.clear()
    username.send_keys("dilkinn@gmail.com")
    time.sleep(2)

    password = driver.find_element_by_name("user[password]")
    password.clear()
    password.send_keys("pa4a4ok73")
    time.sleep(2)

    driver.find_element_by_xpath("//input[contains(@value,'Sign in')]").click()
    time.sleep(2)
    return driver

In [5]:
## book_links

def scrap_book_links(page1):
    driver = webdriver.Chrome('chromedriver')
    driver = gr_sign_in(driver)
    arr_all = []
    
    driver.get(page1)
    time.sleep(1)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    arr = get_book_links_from_soup(soup)
    arr_all.extend(arr)
    
    while True:
        try:
            print("next_page")
            driver.find_element_by_xpath("//a[contains(@rel,'next')]").click()
            time.sleep(1)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            arr = get_book_links_from_soup(soup)
            arr_all.extend(arr)           
        except:
            driver.close()
            break
            
    print(len(arr_all))
    np.array(arr_all)
    df = pd.DataFrame(arr_all)
    df.to_csv(f'../data/book_links_goodreads.csv')
    return df

def get_book_links_from_soup(soup):
    b_links = []
    divs = soup.find_all('a', {"class": "leftAlignedImage"})
    for div in divs:
        b_links.append('http://goodreads.com' + div.get('href'))
    return b_links

In [6]:
# popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science'
# book_links_df = scrap_book_links(popular_science_link)

In [7]:
book_links_df = pd.read_csv('../data/book_links_goodreads.csv')

In [8]:
b_l = list(book_links_df['0'])

In [9]:
b_l[99]

'http://goodreads.com/book/show/26150770-the-big-picture'

In [10]:
b_l_1 = b_l

In [11]:
for b in b_l.copy():
    if 'author' in b:
        b_l_1.remove(b)

for b in b_l_1:
    if 'author' in b:
        print(b)

In [13]:
len(b_l_1)

1250

# Get information about the books

In [111]:
def get_the_soup(book_link):
    driver.get(book_link)
    time.sleep(5)
    soup = bs(driver.page_source, 'html.parser')
    print('soup ok')
    return soup

def get_book_info(book_link, soup):
    try:
        isbn_str = soup.find_all('div',{"class":"infoBoxRowItem"})[1].get_text()

        isbn_str = isbn_str.replace('\n', '')
        isbn_str = isbn_str.replace('(', '')
        isbn_str = isbn_str.replace(')', '')
        isbn, isbn13 = isbn_str.strip().split()[0], isbn_str.strip().split()[2]
        print('isbn ok')
    except:
        print('get_book_info: isbn failed')
        isbn, isbn13 = '0', '0'

    try:
        title_str = soup.find_all('h1',{"id":"bookTitle"})[0].get_text()
        title_str = title_str.replace('\n', "").strip()
        print('title ok')
    except:
        print('get_book_info: title failed')
        title_str = ""
        
    try:
        original_title = soup.find_all('div',{"id":"bookDataBox"})[0].find_all('div',{'class':'infoBoxRowItem'})[0].get_text()
        print('original title ok')
    except:
        print('get_book_info: original titile failed')
        original_title = '0'
    
    try:
        avg_rating = soup.find_all('span',{"itemprop":"ratingValue"})[0].get_text()
        avg_rating = avg_rating.replace('\n', '').strip()
        avg_rating = float(avg_rating)
        print('avg rating ok')
    except:
        print('get_book_info: rating failed')
        avg_rating = 0
    
    
    try:
        author_link = soup.find_all('a',{"class":"authorName"})[0].get('href')
        print('author_link ok')
    except:
        author_link = ''
        print('get_book_info: author link failed')

    authors_names = []
    try:
        spans = soup.find_all('a',{"class":"authorName"})[0].find_all('span',{'itemprop':'name'})
        for span in spans:
            authors_names.append(span.get_text())
        print('author names ok')
    except:
        authors_names = authors_names
        print('author names failed')
    
    try:
        book_description = soup.find_all('div',{"id":"description"})[0].find_all('span')[1].get_text()
        print('book description ok')
    except:
        desc = '0'
        print('book description failed')
    
    dct = {'book_link': book_link, 'isbn': isbn, 'isbn13': isbn13, 
          'book_title' : title_str, 'original_title' : original_title,
          'avg_rating' : avg_rating, 'author_link': author_link,
          'author_names': authors_names, 'book_description': book_description}
    print('get book info ok')
    return dct

In [104]:
def get_reviews_array_from_soup(soup):
    arr = []
    divs = soup.find_all('div',{"id":"bookReviews"})[0].find_all('div',{'class':'friendReviews elementListBrown'})
    
    for div in divs:
        user_link = 'http://goodreads.com' + div.find_all('span',{'itemprop':'author'})[0].find('a').get('href')
        ranking = len(div.find_all('span', {'class':'staticStar p10'}))
        try:
            review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[1].get_text()
        except:
            review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[0].get_text()
        dct = {"isbn":isbn, 'user_link': user_link, 'ranking':ranking, "review": review}
        arr.append(dct)
    print('get reviews array from soup ok')
    return arr

def get_reviews_from_book_link(isbn, b_l):
    arr_all = []
    
    driver.get(b_l)
    time.sleep(10)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    arr = get_reviews_array_from_soup(soup)
    
    if (len(arr) <= 30):
        time.sleep(5)
        html = driver.page_source
        soup = bs(html, 'html.parser')
        arr = get_reviews_array_from_soup(soup)
    arr_all.extend(arr)
    
    while True:
        try:
            driver.find_element_by_xpath("//a[contains(@rel,'next')]").click()
            time.sleep(10)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            arr = get_reviews_array_from_soup(soup)
            if (len(arr) <= 30):
                time.sleep(5)
                html = driver.page_source
                soup = bs(html, 'html.parser')
                arr = get_reviews_array_from_soup(soup)
            arr_all.extend(arr)         
        except:
#             driver.close()
            print('exception')
            break
            
    if len(arr_all) > 270:
        print('succeed')
    else:
        print('failed')
    df = pd.DataFrame(arr_all)
    df.to_csv(f'../data/revs_{isbn}.csv')
    print('reviews file created')
    return df

In [22]:
len(b_l)

1250

In [23]:
books_arr = []
failed_books_list = []
# driver = webdriver.Chrome('chromedriver')
# driver = gr_sign_in(driver)

chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = gr_sign_in(driver)

for n in range(0, 1200, 100):
    for i, book_l in enumerate(b_l_1[n:n+100]):
        sys.stdout.write("\r" + f'book n : {i}\n')
        sys.stdout.flush()
        try:
            print('before soup')
            print(book_l)
            soup = get_the_soup(book_l)
            print('after soup')
            dct = get_book_info(book_l, soup)
            isbn = dct['isbn']
            if len(isbn) == 10:
                reviews = get_reviews_from_book_link(isbn,book_l)
                books_arr.append(dct)
                df = pd.DataFrame(books_arr)
                df.to_csv(f'../data/gr_books_df_{n}.csv')
            else:
                failed_books_list.append(bool_l)
        except:
            failed_books_list.append(book_l)
            continue

book n : 0
before soup
http://goodreads.com/book/show/3869.A_Brief_History_of_Time
soup ok
after soup
exception
failed
book n : 1
before soup
http://goodreads.com/book/show/21.A_Short_History_of_Nearly_Everything
soup ok
after soup
exception
failed
book n : 2
before soup
http://goodreads.com/book/show/61535.The_Selfish_Gene
soup ok
after soup
exception
failed
book n : 3
before soup
http://goodreads.com/book/show/23692271-sapiens
soup ok
after soup
book n : 4
before soup
http://goodreads.com/book/show/55030.Cosmos
soup ok
after soup
exception
failed
book n : 5
before soup
http://goodreads.com/book/show/63697.The_Man_Who_Mistook_His_Wife_for_a_Hat_and_Other_Clinical_Tales
soup ok
after soup
exception
failed
book n : 6
before soup
http://goodreads.com/book/show/21413662-what-if-serious-scientific-answers-to-absurd-hypothetical-questions
soup ok
after soup
exception
failed
book n : 7
before soup
http://goodreads.com/book/show/1842.Guns_Germs_and_Steel
soup ok
after soup
exception
failed
bo

In [23]:
b_l_1[0]

'http://goodreads.com/book/show/3869.A_Brief_History_of_Time'

In [24]:
driver.get(b_l_1[0])

In [26]:
# driver = webdriver.Chrome('chromedriver')
# driver = gr_sign_in(driver)

chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = gr_sign_in(driver)

n=1200
for i, book_l in enumerate(b_l_1[n:n+50]):
    sys.stdout.write("\r" + f'book n : {i}\n')
    sys.stdout.flush()
    try:
        print('before soup')
        print(book_l)
        soup = get_the_soup(book_l)
        print('after soup')
        dct = get_book_info(book_l, soup)
        isbn = dct['isbn']
        if len(isbn) == 10:
            reviews = get_reviews_from_book_link(isbn,book_l)
            books_arr.append(dct)
            df = pd.DataFrame(books_arr)
            df.to_csv(f'../data/gr_books_df_{n}.csv')
        else:
            failed_books_list.append(bool_l)
    except:
        failed_books_list.append(book_l)
        continue

book n : 0
before soup
http://goodreads.com/book/show/208743.The_Millennium_Problems
soup ok
after soup
exception
failed
book n : 1
before soup
http://goodreads.com/book/show/64838.Alpha_and_Omega
soup ok
after soup
exception
failed
book n : 2
before soup
http://goodreads.com/book/show/45995328-de-meeste-mensen-deugen
soup ok
after soup
exception
succeed
book n : 3
before soup
http://goodreads.com/book/show/36556202-the-coddling-of-the-american-mind
soup ok
after soup
book n : 4
before soup
http://goodreads.com/book/show/48890486-breath
soup ok
after soup
book n : 5
before soup
http://goodreads.com/book/show/52668915-entangled-life
soup ok
after soup
book n : 6
before soup
http://goodreads.com/book/show/33916545-beyond-infinity
soup ok
after soup
book n : 7
before soup
http://goodreads.com/book/show/38728977-the-molecule-of-more
soup ok
after soup
book n : 8
before soup
http://goodreads.com/book/show/50263828-this-book-could-save-your-life
soup ok
after soup
book n : 9
before soup
http

In [32]:
print(len(failed_books_list))
print(len(new_fail_list))

328
328


In [105]:
def try_another_edition(soup):
    dct ={}
    divs = soup.find_all('div',{"class":"otherEdition"})
    links = []
    for div in divs:
        link = div.find('a').get('href')
        links.append(link)
    for link in links:
        print(link)
        try:
            soup = get_the_soup(link)
            dct = get_book_info(book_l, soup)
            isbn = dct['isbn']
            if len(isbn) == 10:
                print('success')
                reviews = get_reviews_from_book_link(isbn,book_l)
                books_arr.append(dct)
                df = pd.DataFrame(books_arr)
                df.to_csv(f'../data/gr_books_df_failed.csv')
                break
            else:
                print('isbn fail')
        except:
            print('fail')
    return 

In [112]:
new_fail_list = []
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = gr_sign_in(driver)

n=1200
for i, book_l in enumerate(failed_books_list):
    sys.stdout.write("\r" + f'book n : {i}\n')
    sys.stdout.flush()
    try:
        soup = get_the_soup(book_l)
        dct = get_book_info(book_l, soup)
        isbn = dct['isbn']
        desc = dct['book_description']
        book_title = dct['book_title']
        avg_rating = dct['avg_rating']
        if (len(isbn) != 10) or (len(desc) == 0) or (len(book_title) == 0) or (avg_rating == 0):
            print(f'isbn is broken : {isbn}')
            try_another_edition(soup)
        else:
            new_fail_list.append(bool_l)
    except:
        new_fail_list.append(book_l)
        continue

book n : 0
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn is broken : 0
https://www.goodreads.com/book/show/20873740-sapiens
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn fail
https://www.goodreads.com/book/show/23198201-sapiens
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn fail
https://www.goodreads.com/book/show/23346740-sapiens
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn fail
https://www.goodreads.com/book/show/20839196-sapiens
soup ok
isbn ok
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
success
get reviews array f

In [113]:
len(new_fail_list)

69

In [107]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = gr_sign_in(driver)

In [108]:
book_l = new_fail_list[0]
soup = get_the_soup(book_l)
# dct = get_book_info(book_l, soup)
# isbn = dct['isbn']

soup ok


In [ ]:
book_description = soup.find_all('div',{"id":"description"})[0].find_all('span')[1].get_text()

In [110]:
authors_names = []
spans = soup.find_all('a',{"class":"authorName"})[0].find_all('span',{'itemprop':'name'})
for span in spans:
    authors_names.append(span.get_text())